## Important information
<font color="red"> <b> ⚠ This documment is written as a Jupyter notebook and the code used to produce these analyses may be hidden for ease of readability. The notebook's interactivity as an exported html file is limited. For full widgets-interactivity it needs to run with a python kernel (e.g. jupyter notebook, jupyterlab apps) <br>
    To make the code visible click here:
 </b> </font>

In [ ]:
%%HTML
<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Hide/show code"></form>

# Run data preprocessing

In [ ]:
%%capture
%run 01_Data_preprocessing.ipynb

# FFT Functions

In [ ]:
# Fast Fourier Transform
def myFFT(x,y,Ts):
    Fs = 1/Ts;  # sampling rate
    x = np.arange(0,1,Ts) # time vector
    n = len(y) # length of the signal
    k = np.arange(n)
    X = Fs*k/n # two sides frequency range
    X = X[0:int(n/2)] # one side frequency range
    Y = np.fft.fft(y)/n # fft computing and normalization
    Y = Y[0:int(n/2)]
    return X, np.abs(Y), Y

# Cubic spline interpolation
def myinterpolation(x, y, num=None):
    if num is None:
        num=len(x)
    f = interp1d(x, y, kind='cubic')
    xnew = np.linspace(x[0], x[-1], num=num, endpoint=True)
    ynew = f(xnew)
    return xnew, ynew 

In [ ]:
def applyFFT(Expe):
    for E in Expe:
            # encoder 1
        [E.ts, E.rpms_1] = myinterpolation(E.t1a, E.rpm_1)
        [E.fft_freq_1, E.fft_amp_1, E.fft_1] = myFFT(E.ts, E.rpms_1,E.Ts)
            # encoder 2
        [E.ts, E.rpms_2] = myinterpolation(E.t2a,E.rpm_2)
        [E.fft_freq_2, E.fft_amp_2, E.fft_2] = myFFT(E.ts, E.rpms_2,E.Ts)
            # encoder 1 minus 2
        E.rpms_12 = E.rpms_1 - E.rpms_2
        [E.fft_freq_1minus2, E.fft_amp_1minus2, E.fft_1minus2] = myFFT(E.ts, E.rpms_12,E.Ts)

applyFFT(HeS)
applyFFT(CrS)

# Frequency spectrum
In order to compute a classical fast discrete Fourier transform, the angular-sampled data needs to be interpolated with regular time basis. The resampling is done using a cubic spline interpolation and the number of data points is kept from the original data. The interpolation code is based on the provided library from the Scilab [interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) module.

As each experiment contains encoder data for different number of revolutions, and because the average target angular velocities differ by an order of magnitude between the first and the last data sets, the measurement durations $T_{f_i}$ also differ. Accordingly, the frequency resolution varies with $\frac{1}{T_{f_i}}$ and so the resolution at higher angular velocities gets lower.

In [ ]:
def create_df(attrs, encoders='all', start_index=1, end_index=-1, sort_key=None):
    if encoders=='all':
        encoders = ['1', '2', '1minus2']
    df_list = []
    for health, Expe in zip(['healthy', 'cracked'], [HeS, CrS]):
        for E in Expe:
            for n in encoders:
                var_list = [getattr(E, f'{at}_{n}')[start_index:end_index] for at in attrs.values()] 
                col_names= [at for at in attrs.keys()]
                df = pd.DataFrame(data=np.array(var_list).T, columns=col_names)
                df['encoder'] = n
                df['rpm'] = E.speed_rpm_num
                df['series'] = E.speed_rpm_str
                df['health'] = health
                if sort_key is not None:
                    df.sort_values(sort_key, inplace=True)
                df_list.append(df)
    return pd.concat(df_list)

In [ ]:
def showdf(df, inputs=None, filters=None, title=None):
    fig_height = 700
    fig_width = None
    fig  = go.FigureWidget(layout_height=fig_height, layout_width=fig_width)
    df_filt = df.copy()

    num_keys = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['float64', 'int64']]
    obj_keys = [key for key in dict(df.dtypes) if dict(df.dtypes)[key] in ['O']]
    options = [None] + list(df.keys())


    wd = dict(
        x = widgets.Dropdown(description='x', options=options),
        y = widgets.Dropdown(description='y', options=options),
        z = widgets.Dropdown(description='z', options=options),
        color = widgets.Dropdown(description='color', options=[None] + obj_keys),
        line_group = widgets.Dropdown(description='group', options = [None] + obj_keys),
        facet_row = widgets.Dropdown(description='row', options=[None] + obj_keys),
        facet_col = widgets.Dropdown(description='column', options=[None] + obj_keys),
        facet_col_wrap = widgets.IntText(description='wrap', min=0, max=10, value=6, layout=dict(width='70px'), style=dict(description_width='30px')),
        #animation_frame = widgets.Dropdown(description='animation', options=[None] + obj_keys),
        log_x = widgets.Checkbox(description='log', layout=dict(width='auto'), style=dict(description_width='0px')),
        log_y = widgets.Checkbox(description='log', layout=dict(width='auto'), style=dict(description_width='0px')),
        log_z = widgets.Checkbox(description='log', layout=dict(width='auto'), style=dict(description_width='0px')),
        update = widgets.Checkbox(description='update plot', value=False)
    )
    if inputs is not None:
        for k,v in inputs.items():
            if k in wd:
                wd[k].value = v

    fw = dict()
    for k in df.keys():
        if k in num_keys:
            opts = [df[k].min(), df[k].max()]
            if filters is not None:
                val = filters[k] if k in filters else opts
            else:
                val = opts
            fw[k] = widgets.FloatRangeSlider(min= opts[0], max=opts[1], value=val, continuous_update=False)
        elif k in obj_keys:
            opts = list(df[k].unique())
            val = opts
            if filters is not None:
                if k in filters:
                    v = filters[k]
                    val = v if isinstance(v,(list,tuple)) else [v]
                
            fw[k] = widgets.SelectMultiple(value=val, options=opts, rows=5 if len(opts)>=5 else len(opts))
                    
    def update_fig(**params):
        nonlocal df_filt
        if wd['update'].value==True:
            if not params:
                params = {k:v.value for k,v in wd.items()}
            update = params['update']
            p = params.copy()
            p.pop('update')
            if p['z'] is None:
                fn = px.line
                p.pop('z')
                p.pop('log_z')
            else:
                fn = px.line_3d
                p.pop('facet_col')
                p.pop('facet_row')
                p.pop('facet_col_wrap')
            pxfig = fn(df_filt, height=fig_height, width=fig_width, render_mode='svg', **p)
            fig.data = []
            fig.add_traces(pxfig.data)
            pxfig.layout.margin.t=20
            fig.layout = pxfig.layout

    def update_filters(**params):
        nonlocal df_filt
        if not params:
            params = {k:v.value for k,v in fw.items()}
        df_filt=df.copy()
        for k,v in params.items():
            if k in num_keys:
                df_filt = df_filt[(df_filt[k]>=v[0]) & (df_filt[k]<=v[1])]
            elif k in obj_keys:
                df_filt = df_filt[df_filt[k].isin(v)]
            
        if wd['update'].value == True:
            update_fig()

    df_fw = widgets.interactive(update_filters, **fw)
    df_wd = widgets.interactive(update_fig, **wd)
    wd['update'].value = True
    
    acc_wd = widgets.Accordion([widgets.VBox([widgets.HBox([wd['x'],wd['log_x']]),
                                              widgets.HBox([wd['y'],wd['log_y']]),
                                              widgets.HBox([wd['z'],wd['log_z']]),
                                              wd['color'],
                                              wd['line_group'],
                                              wd['facet_row'],
                                              widgets.HBox([wd['facet_col'], wd['facet_col_wrap']]),
                                              wd['update']
                                             ])])
    acc_wd.set_title(0,'Graph parameters')
    acc_fw = widgets.Accordion([widgets.VBox([df_fw])])
    acc_fw.set_title(0,'Dataframe filters')
    accs = widgets.VBox([acc_wd, acc_fw])
    fig_acc = widgets.Accordion([fig])
    fig_acc.set_title(0,'Vizualization: '+ title)
    return widgets.HBox([accs, fig_acc])

inputs = dict(x='frequency', y='amplitude', color='encoder', line_group='series',log_y=True, facet_row='health', facet_col='encoder')
filters = dict(frequency=[0,50])

In [ ]:
df_downsampled = create_df(attrs=dict(frequency='fft_freq', amplitude='fft_amp'), sort_key='frequency')

showdf(df_downsampled, inputs, filters, title = 'FFT on downsampled encoder signals')

# Spectrogram of the adapted datasets

## Resampling

As mentioned before the dataset times for each experiment are varying. In order to compute a spectrogram we need to cut the datasets times  according to the shortest one and all the other datasets which are longer will be truncated to this time. In terms of indexes for $T_f=$ 9 secs and $rpm_i$ ranging from 100rpm to 3000rpm, it corresponds to an index value of $rpm_i \cdot \frac{imp}{60 \cdot D_i} \cdot T_f$. The calculated values are shown in the following plot.

In [ ]:
HeS.time_steps = HeS.cut_time/HeS.cut_indexes
CrS.time_steps = CrS.cut_time/CrS.cut_indexes

In [ ]:
def plot_reeindexing(Expe):
    fig  = make_subplots(shared_xaxes=True, rows=2,cols=1, vertical_spacing=0.05)
    fig.add_bar(x=Expe.speed_list, y=Expe.cut_indexes, showlegend=False,
                row=1, col=1
               )
    fig.add_bar(x=Expe.speed_list, y=Expe.time_steps*1000, showlegend=False,
                name='Time steps [µs]',
                row=2,col=1
               )
    fig.update_layout(
        xaxis2_title = 'Driven angular velocity',
        yaxis_title = 'Index count',
        yaxis2_title = 'Time steps [ms]',
        title_text = f'Number of indices and time steps for {Expe.cut_time:d} seconds cut-off time',
        height = 400
    )
    fig.show()
    save_plot_button(fig)

In [ ]:
plot_reeindexing(HeS)

In order to get compatible array dimensions, the signals will be interpolated and resampled to the maximum number of the list (5570 samples) corresponding new time step value of $T_s =$ 1562µs

In [ ]:
def applyFFTsync(Expe):
    NF = max(Expe.cut_indexes)
    for i in range(Expe.numoffiles):
        E = Expe[i]
        E.Ts = Ts = Expe.cut_time/NF
        N = Expe.cut_indexes[i]
            # encoder 1
        [E.ts, E.rpms_cut_1] = myinterpolation(E.t1a[0:N], E.rpms_1[0:N], NF)
        [E.fft_cut_freq_1, E.fft_cut_amp_1, E.fft_cut_1] = myFFT(E.ts, E.rpms_cut_1,Ts)
            # encoder 2
        [E.ts, E.rpms_cut_2] = myinterpolation(E.t2a[0:N],E.rpms_2[0:N], NF)
        [E.fft_cut_freq_2, E.fft_cut_amp_2, E.fft_cut_2] = myFFT(E.ts, E.rpms_cut_2,Ts)
            # encoder 1 minus 2
        E.rpms_cut_1minus2 = E.rpms_cut_1 - E.rpms_cut_2
        [E.fft_cut_freq_1minus2, E.fft_cut_amp_1minus2, E.fft_cut_1minus2] = myFFT(E.ts, E.rpms_cut_1minus2,Ts)
        E.ts_1 = E.ts_2 = E.ts_1minus2 = E.ts

applyFFTsync(HeS)
applyFFTsync(CrS) 

In [ ]:
df_signalsync = create_df(attrs=dict(rpm_speed='rpms_cut', time='ts'), encoders=['1minus2'], sort_key='time')

inputs = dict(x='time', y='rpm_speed', color='encoder', facet_row='health')
filters = dict(time=[0,1], series='100rpm')
showdf(df_signalsync, inputs, filters, title='Synced time-based signals')

## Frequency spectrogram
The 3 dimensional spectrogram regroups the frequency spectra for separated driven angular velocities of the motor from each experiment measurements in a single figure.

In [ ]:
def plotFFT3D(Expe, fstart = 1, fend = None, log=True):
    cut_time = Expe.cut_time
    fstart = fstart # [Hz]
    if fend == None:
        fend = 1.5*Expe.f # [Hz]
    
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    
    x = np.linspace(Nstart/cut_time,Nend/cut_time,Nend-Nstart)
    y = Expe.speeds    

    z_data1=[]
    z_data2=[]
    z_data12=[]

    for i in range(Expe.numoffiles):
        z_data1.append(Expe[i].fft_cut_amp_1[Nstart:Nend])
        z_data2.append(Expe[i].fft_cut_amp_2[Nstart:Nend])
        z_data12.append(Expe[i].fft_cut_amp_1minus2[Nstart:Nend])

    fig = make_subplots(rows=1, cols=3,
                        specs=[[{"type": "surface"}]*3],
                        print_grid=False,
                        subplot_titles = ('encoder 1', 'encoder 2', 'encoder difference')
                       )
    fig.append_trace(dict(type='surface', x=x, y=y, z=np.log(z_data1) if log else z_data1,name='enc1', 
                          showscale=False,colorscale='Viridis', reversescale=True), 1, 1)
    fig.append_trace(dict(type='surface', x=x, y=y, z=np.log(z_data2) if log else z_data2,name='enc2', 
                          showscale=False,colorscale='Viridis', reversescale=True), 1, 2)
    fig.append_trace(dict(type='surface', x=x, y=y, z=np.log(z_data12) if log else z_data12,name='enc1-2', 
                          showscale=False,colorscale='Spectral', reversescale=True), 1, 3)

    scene=dict(xaxis=dict(title='Freq [Hz]',titlefont_size=10, tickfont_size=10),
               yaxis=dict(title='Speed [rpm]',titlefont_size=10, tickfont_size=10),
               zaxis=dict(title='Amp',titlefont_size=10, tickfont_size=10),
               camera_eye = {'x': 1.8, 'y': 1, 'z': 1.8}
              )
    figtitle = f'Frequency 3D-spectrograms for respectively encoder 1, encoder 2 and encoder difference<br>' + Expe.shaft_desc
    fig.layout = go.Layout( width=1000,
        title=figtitle,
        scene1=scene,
        scene2=scene,
        scene3=scene,
        margin = dict(l=20,r=20)
    )


    fig.show()

    #save_plot_button(fig)

## Healthy shaft

In [ ]:
plotFFT3D(HeS)

## Cracked shaft

In [ ]:
plotFFT3D(CrS)

## Comparison of cracked and healthy shaft (encoder difference 1-2)

In [ ]:
def plotFFT3Dboth(Expea, Expeb, fstart = 1, fend = None, log=True):
    cut_time = Expea.cut_time
    fstart = fstart # [Hz]
    if fend == None:
        fend = 1.5*Expea.f # [Hz]
    
    Nstart = int(fstart*cut_time)
    Nend = int(fend*cut_time)
    
    x = np.linspace(Nstart/cut_time,Nend/cut_time,Nend-Nstart)
    y = Expea.speeds    

    z_data12a=[]
    z_data12b=[]

    for i in range(Expea.numoffiles):
        z_data12a.append(Expea[i].fft_cut_amp_1minus2[Nstart:Nend])
        z_data12b.append(Expeb[i].fft_cut_amp_1minus2[Nstart:Nend])

    fig = go.Figure()
    fig.add_trace(dict(type='surface', x=x, y=y, z=np.log(z_data12a) if log else z_data12a, name='enc12_healthy', 
                       colorscale='Viridis', reversescale=True,
                       colorbar=dict(title = 'healthy <br>shaft')
                      )
                 )
    fig.add_trace(dict(type='surface', x=x, y=y, z=np.log(z_data12b) if log else z_data12b, name='enc12_cracked', 
                       colorscale='hot',
                       colorbar=dict(x=1.2, title = 'cracked <br>shaft')
                       )
                 )

    fig.layout.scene=dict(xaxis=dict(title='Freq [Hz]',titlefont_size=10, tickfont_size=10),
               yaxis=dict(title='Speed [rpm]',titlefont_size=10, tickfont_size=10),
               zaxis=dict(title='Amp',titlefont_size=10, tickfont_size=10),
               camera_eye = {'x': 1.4, 'y': 1, 'z': 1.4}
              )
    fig.layout.height = 450
    fig.layout.title.text = f'Frequency 3D-spectrograms comparison between healthy and cracked shaft <br> encoder difference'
    
    fig.show()
    
    #save_plot_button(fig)
    

In [ ]:
plotFFT3Dboth(HeS, CrS)

The multiples of the motor rotational frequency are forming a series of spikes aligned on different diagonals in the frequency-speed plane and can be seen in the 3D plot above. The natural frequency of the shaft at around **33Hz** (**21Hz** for the cracked shaft) is also clearly to be seen for almost all the different angular velocities. It is however to be noticed that the amplitudes at the expected natural frequency are the highest when the this value corresponds to an integer multiple of the shaft rotational speed (see table in next section). In these cases the motor is exciting the system which gets into some resonance mode. Additionally the measured time window of **9s** only allows a precision in the frequency domain of **0.11Hz**